In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [ ]:
# the project is under progress , this is just a first structure of my work on it.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
df = pd.read_csv(filepath_or_buffer='../input/capstone-project-on-zomato/Zomato Bangalore Data.csv',encoding='latin1')

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
len(df)

In [ ]:
# separating 4.2/5/0 to 4.2 and 5.0 an accepting only first split 
df['rate']=df['rate'].str.split("/",n=1,expand=True)[0]

In [ ]:
#heatmap to view any null values in the dataframe " df" and white lines in the graph shows missing values at an index
plt.figure(figsize=(10,9))
sns.set_context('notebook')
sns.heatmap(df.isnull(),yticklabels=False)
plt.tight_layout()

In [ ]:
df.columns

In [ ]:
#df[ (df['rate']=='NEW') & (df['reviews_list']=='[]')].count()

## Fixing rate column

In [ ]:
import re 

In [ ]:
new_data=df.copy()

In [ ]:
len(new_data[new_data["reviews_list"]=="[]"])

In [ ]:
# finding all the index where review is empty text 
index_MissingReview = new_data[new_data["reviews_list"]=="[]"].index
print(index_MissingReview)

In [ ]:
#new_data.drop(index_MissingReview,inplace=True)

In [ ]:
len(new_data)

> #### since there are some missing rate values , in order to fix them i am trying to fetch rating from Review column using regex and take average of it.
###### **The below code is fetching the text with Rated 3.0 like text and remove "rated " from it and taken the average of the float values left **

In [ ]:
rating_from_reviews =[]
for sentences in new_data.reviews_list:
    patterns= r"Rated\s\d"
    text = re.findall(patterns,sentences)
    if len(text)==0:
        rating_from_reviews.append(round((sum(text)/(len(text)+1)),1))
    else:
        text= [phrase.split(' ')[1] for phrase in text]
        text = list(map(int, text))
        rating_from_reviews.append(round((sum(text)/len(text)),1))

In [ ]:
len(rating_from_reviews)

In [ ]:
rating_from_reviews[1]

In [ ]:
rate_dict={i : ele for i,ele in enumerate(rating_from_reviews)} # converting list into dictonary

In [ ]:
len(rate_dict)

In [ ]:
new_data.loc[new_data["rate"]=="NEW","rate"]=np.nan # replacing all "NEW" values in rate column to null

In [ ]:
new_data.isnull().sum()

In [ ]:
#fixing null values in rate with rate_dict)

In [ ]:
new_data['rate'].fillna(rate_dict,inplace=True)

In [ ]:
#new_data.isnull().sum()

In [ ]:
#changing type to float 
new_data['rate']=new_data['rate'].astype(float) 

In [ ]:
working_data=new_data[['reviews_list','rate']] #working_data dataframe only includes columns required for sentiment analysis

In [ ]:
working_data.shape

## Cleaning review columns 

### Task -1 removing blank reviews

In [ ]:
#finding all th blank text from new_data['review_text'] eg : ["  '  '"]
string1 = "h      \\\\\ 2*2"
index_noText=[]
for i , j in enumerate(working_data['reviews_list']):
    if not re.search(r'^.*[A-Za-z].*$', j):
        index_noText.append(i)

In [ ]:
len(index_noText)

In [ ]:
# remove all the rows with now review_text
new_data.drop(index_noText,axis=0,inplace=True)

In [ ]:
#due to deletion of some rows , there were some missing index number , hence reset . 
new_data.reset_index(drop = True,inplace=True)

In [ ]:
#check if there is any blank review 
from numpy import random
arr = random.randint(new_data.shape[0]) # picking random values b/w 0 and 18954
#print(arr,"\n",new_data.reviews_list[arr]) #( to print the desired outcome , keep checking if there is any blank review )

#### Task -1 done 

#### Task -2 removing " Rated 3.0 and RATED n"  

In [ ]:
clean_reviews1=[] #removing rated 3.0 or rated n
patterns=r"(Rated\s+\d.\d.\W)|(RATED)|(\\n)|(\d/\d)"
for i,word in enumerate(working_data['reviews_list']):
    text= re.sub(patterns,"",word)
    clean_reviews1.append(text)

In [ ]:
if (len(clean_reviews1)==working_data.shape[0]): print("TRUE")

In [ ]:
#check if there is any review with rated or RATED
from numpy import random
arr = random.randint(working_data.shape[0]) # picking random values between 0 and 18954
#print(arr,'\n',clean_reviews1[arr])#( to print the desired outcome )

#### Task -2 END

### Task -3  : Change all words into lower case

In [ ]:
change1=list(map(lambda x: x.lower(),clean_reviews1))

In [ ]:
if (len(change1)==working_data.shape[0]): print("TRUE")

In [ ]:
#check if there is any review with rated or RATED
from numpy import random
arr = random.randint(working_data.shape[0]) # picking random values between 0 and 18954
print(arr,'\n',change1[arr]) #( to print the desired outcome )

### Task - 3 ends

### Task-4 starts , removal of puncuation except  { , and '  } 

In [ ]:
import string 

In [ ]:
punc = string.punctuation
# punc (result : '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')

In [ ]:
#all the puncation which i dont require as of now ( if required )
punc = punc.replace(",", "")
punc = punc.replace("'", "")
punc = punc.replace(".", "")


In [ ]:
remove_punc=r'[{}]'.format(re.escape(punc))
# remove_punc ( result :'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]') without any removal of specific puncuation 

In [ ]:
clean_reviews2=[]
for i, j in enumerate(change1):
    clean_reviews2.append(re.sub(remove_punc," ",j))

In [ ]:
if (len(clean_reviews2)==working_data.shape[0]): print("TRUE")

In [ ]:
#check if there is any puncuation or not except ( , . '')
from numpy import random
arr = random.randint(working_data.shape[0]) # picking random values between 0 and 18954
#print(arr,'\n',clean_reviews2[arr]) #( to print the desired outcome )

### Task - 4 ends 

### Task -5 Removal of Emoji 

In [ ]:
import emoji

In [ ]:
clean_reviews3=[]
for i, word in enumerate(clean_reviews2):
    clean_reviews3.append(emoji.replace_emoji(word,""))

In [ ]:
if (len(clean_reviews3)==working_data.shape[0]): print("TRUE")

In [ ]:
#check if there is any emoji
from numpy import random
arr = random.randint(working_data.shape[0]) # picking random values between 0 and 58498
#clean_reviews3[1] #( to print the desired outcome )

In [ ]:
#find if there is any emoji left in the clean_review6
for i, j in enumerate(clean_reviews3):
    if emoji.emoji_count(j) >0:
        print(i)
    

> ### **no outcome from above code, confirms no emoji in any review in review_list**

### Task -5 ends 

### Task - 6 " removal of not neeeded text "
###### \x83ã\x83ã\x82ã\x83ã\x83ã\x82ã\x82ã\x83ã\x83ã\x83ã\

In [ ]:
clean_reviews4=[]
ph = r"['\x83ã\x83ã\x82ã\x83ã\x83ã\x82ã\x82ã\x83ã\x83ã\x83ã\x82ã\x82ã\x83ã\x82ã\x82ã¢ã\x83ã\x83ã\x82ã\x83ã\x83ã\x82ã\x82ã\x82ã\x83ã\x83ã\x82ã\x82ã\x83ã\x82ã\x82ã\x80ã\x83ã\x83ã\x82ã\x83ã\x83ã\x82ã\x82ã\x82ã\x83ã\x83ã\x82ã\x82ã\x83ã\x82ã\x82ã']|[â]"
for i, j in enumerate(clean_reviews3):
    text=re.sub(ph,"  ",j)
    clean_reviews4.append(re.sub(r"\s{2}","",text))
#clean_reviews4[1]

In [ ]:
if (len(clean_reviews4)==working_data.shape[0]): print("TRUE")

In [ ]:
#check if there is above text 
from numpy import random
arr = random.randint(working_data.shape[0]) # picking random values between 0 and 18954
print(arr,'\n',clean_reviews4[arr]) 
#errors #( to print the desired outcome 10277 , 18637 ( #hashtags: hennaichubbychennaigirlchickenwaffleconefriedfriedchickensauceperiperichickenperiperiperibanglorediariesyumbloggerlifeblogbloggerchennaibloggerchennaifoodguidechennaifoodbloggerfoodbloggerfoodforlifewalkwithchennaiwalkwithindiafoodphotographyfoodpicoftheday))

### Task -6 ends

### Task -7 removing all the digits from the reviews

In [ ]:
from string import digits
print(digits)

In [ ]:
clean_reviews5=[]
remove=r'[{}]'.format(digits)
for i, j in enumerate(clean_reviews4):
    clean_reviews5.append(re.sub(remove," ",j))

In [ ]:
if (len(clean_reviews5)==working_data.shape[0]): print("TRUE")

In [ ]:
#check if there is any digit
from numpy import random
arr = random.randint(working_data.shape[0])
print(arr,'\n',clean_reviews5[arr])# picking random values between 0 and 18954

In [ ]:
clean_reviews6=[]
for i, j in enumerate(clean_reviews5):
    clean_reviews6.append(re.sub(r'x\s{1,3}',"",j))

In [ ]:
#check if there is no such patterns are there '(x   x   x   x   x   x   x   )'
from numpy import random
arr = random.randint(working_data.shape[0])
#print(arr,'\n',clean_reviews6[arr])# picking random values between 0 and 18

In [ ]:
#removing pattern such as r'(x   x   x   x   x   x   x   )|(°)|(¤)',clean_reviews5[9749])
#re.sub(r'x\s{1,}',"",clean_reviews5[9749])

### Task -8 , removal of some more unwanted text  

In [ ]:
pattern1=r"(f\s\W\s\W)+"
pattern2=r"(\s°\sf\s\w)|(\s¥\s\¯\s)" 
pattern3=r"(\s\sf\sf\s¼)|(°\sf\s§)|(\s\s°)"
clean_reviews7=[]
for i,j in enumerate(clean_reviews6):
    text=re.sub(pattern1," ",j)
    text=re.sub(pattern2," ",text)
    text=re.sub(pattern3," ",text)
    clean_reviews7.append(text)
#re.findall(pattern1,final_text[18754])#f ° f ¤ ° f f ¼ ° f ¤ ° f f ¼ ( result ['f ¤ ¤', 'f , "', 'f " ,', 'f ¤ °', 'f ¤ °'])
#re.sub(pattern1," ",final_text[18754])
#text_new1 = re.sub(pattern1," ",final_text[18754]) # more requirement ( ° f c),( ¥ ¯ ),(  f  f  f  f)


In [ ]:
if (len(clean_reviews7)==working_data.shape[0]): print("TRUE")

In [ ]:
#check if there it is desired output 
from numpy import random
arr = random.randint(new_data.shape[0]) # picking random values between 0 and 18954
#print(arr,'\n',clean_reviews7[arr]) #( to print the desired outcome )

ntlk stopword : list 
spacy stopwords  : 


### task -9 , #removing whitespaces 

In [ ]:
clean_reviews8=[]
whitespace=r'/s{2}'
punctuation=r'[{}]'.format(string.punctuation)
for i, j in enumerate(clean_reviews7):
    text = re.sub(punctuation," ",j)
    text=re.sub("   "," ",text)
    clean_reviews8.append(text)

In [ ]:
#check if there it is desired output 
from numpy import random
arr = random.randint(working_data.shape[0]) # picking random values between 0 and 18954
#print(arr,'\n',clean_reviews8[arr]) #( to print the desired outcome )

In [ ]:
len(clean_reviews8)

In [ ]:
if (len(clean_reviews8)==new_data.shape[0]): print("TRUE")

In [ ]:
#working_data.drop("cleaned_reviews",axis=1,inplace=True)

In [ ]:
working_data.insert(1,'cleaned_reviews',clean_reviews8)

In [ ]:
working_data.head(3)

## removal of all duplicate reviews 

In [ ]:
len(working_data.drop_duplicates(subset=['cleaned_reviews']))

In [ ]:
no_duplicate_reviews = working_data.drop_duplicates(subset=['cleaned_reviews'])

In [ ]:
len(no_duplicate_reviews)

In [ ]:
no_duplicate_reviews.reset_index(drop = True,inplace=True)

In [ ]:
no_duplicate_reviews.head(3)

In [ ]:
#check if there it is desired output 
from numpy import random
arr = random.randint(no_duplicate_reviews.shape[0]) # picking random values between 0 and 18954
#print(arr,'\n',no_duplicate_reviews.cleaned_reviews[arr],'\n\n',arr-1,'\n',no_duplicate_reviews.cleaned_reviews[arr-1]) #( to print the desired outcome )


## Task -10 ,removal of stop_words

In [ ]:
space_stop_words = nlp.Defaults.stop_words
#print(space_stop_words)
#space_stop_words=="abch" #not , poor, bad ,disappoint,sad,experience,worst,unimpressed

In [ ]:
no_duplicate_reviews['no_stopwords'] = no_duplicate_reviews['cleaned_reviews'].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in space_stop_words))

In [ ]:
no_duplicate_reviews.head(3)

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
# # Download Wordnet through NLTK in python console:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

In [ ]:
lemmitization_output=[]
lemmatizer =WordNetLemmatizer()
for i,word in enumerate(no_duplicate_reviews['no_stopwords']):
    tokens_words = nltk.word_tokenize(word)
    lemmatized_out = ' '.join([lemmatizer.lemmatize(w) for w in tokens_words])
    lemmitization_output.append(lemmatized_out)

In [ ]:
no_duplicate_reviews.insert(3,'lemmitization',lemmitization_output)

In [ ]:
#no_duplicate_reviews.head(3)

In [ ]:
no_duplicate_reviews = no_duplicate_reviews.reindex(columns=['reviews_list', 'cleaned_reviews', 'no_stopwords', 'lemmitization','rate'])
# reordering the column as per the need

In [ ]:
no_duplicate_reviews.head(3)

# Categories the Rating column 

In [ ]:
no_duplicate_reviews['rate'].describe()

In [ ]:
for i in range(1,6):
    print(f"The % of rows with rating less than or equals to {i} is {round(len(no_duplicate_reviews[new_data['rate']<=i])/len(no_duplicate_reviews),3)*100}")
    

In [ ]:
plt.figure(figsize=(12,6))
sns.distplot(no_duplicate_reviews['rate'],color='blue')

In [ ]:
len(no_duplicate_reviews)

In [ ]:
no_duplicate_reviews['rate_cat'] = pd.cut(x=no_duplicate_reviews['rate'], bins=[0,2.5,3.5,4.2,5],
                    labels=['Below Average', 'Average','Above Average','Highly Recommended'
                            ]).astype(str)

In [ ]:
no_duplicate_reviews.groupby('rate_cat')['rate_cat'].count().sort_values(ascending=True)

In [ ]:
#no_duplicate_reviews.head(3)

## Forming Word-Cloud 

In [ ]:
BelowAverage_phrases = no_duplicate_reviews[ no_duplicate_reviews.rate_cat =='Below Average']
BelowAverage_words = []
for t in BelowAverage_phrases.lemmitization:  # Go to the "text" column of "BelowAverage" dataframe
    BelowAverage_words.append(t)

In [ ]:
BelowAverage_text = pd.Series(BelowAverage_words).str.cat(sep = ' ')

In [ ]:
from wordcloud import WordCloud

In [ ]:
# Stopword - Most commonly used word are stopword 
# I want to update list ot stopwords
from wordcloud import ImageColorGenerator
BelowAverage_stop_words = list(space_stop_words) # + ["any addition of words"]

In [ ]:
BelowAverage_wordcloud = WordCloud(stopwords=BelowAverage_stop_words, width = 1600, height = 800, max_font_size = 200, background_color = 'white').generate(BelowAverage_text)


In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(BelowAverage_wordcloud,)
plt.axis("off")

In [ ]:
average_phrases = no_duplicate_reviews[ no_duplicate_reviews.rate_cat =='Average']
average_words = []
for t in average_phrases.lemmitization:  # Go to the "text" column of "neg_phrases" dataframe
    average_words.append(t)

In [ ]:
average_text = pd.Series(average_words).str.cat(sep = ' ')

In [ ]:
average_stop_words = list(space_stop_words) # + ["any addition of words"]

In [ ]:
average_wordcloud = WordCloud(stopwords=average_stop_words, width = 1600, height = 800, max_font_size = 200, background_color = 'white').generate(average_text)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(average_wordcloud)
plt.axis("off")

In [ ]:
aboveAverage_phrases = no_duplicate_reviews[ no_duplicate_reviews.rate_cat =='Above Average']
aboveAverage_words = []
for t in aboveAverage_phrases.lemmitization:  # Go to the "text" column of "neg_phrases" dataframe
    aboveAverage_words.append(t)

In [ ]:
aboveAverage_text = pd.Series(aboveAverage_words).str.cat(sep = ' ')

In [ ]:
aboveAverage_stop_words = list(space_stop_words) # + ["e perience","don t","e","perience","don","t"]

In [ ]:
aboveAverage_wordcloud = WordCloud(stopwords=aboveAverage_stop_words, width = 1600, height = 800, max_font_size = 200, background_color = 'white').generate(aboveAverage_text)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(aboveAverage_wordcloud)
plt.axis("off")

In [ ]:
highlyRecommended_phrases = no_duplicate_reviews[ no_duplicate_reviews.rate_cat =='Highly Recommended']
highlyRecommended_words = []
for t in highlyRecommended_phrases.lemmitization:  # Go to the "text" column of "neg_phrases" dataframe
    highlyRecommended_words.append(t)

In [ ]:
highlyRecommended_text = pd.Series(highlyRecommended_words).str.cat(sep = ' ')

In [ ]:
highlyRecommended_stop_words = list(space_stop_words) # + 

In [ ]:
highlyRecommended_wordcloud = WordCloud(stopwords=highlyRecommended_stop_words, width = 1600, height = 800, max_font_size = 200, background_color = 'white').generate(highlyRecommended_text)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(highlyRecommended_wordcloud)
plt.axis("off")

### Modelling 

In [ ]:
from sklearn.feature_extraction import text

### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

msg_train, msg_test, label_train, label_test = \
train_test_split(no_duplicate_reviews['lemmitization'], no_duplicate_reviews['rate_cat'], test_size=0.2)

print(f"msg_train\t{len(msg_train)}\nmsg_test\t{len(msg_test)}\nlabel_train\t{len(label_train)}\nlabel_test\t{len(label_test)}")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', CountVectorizer(ngram_range=(2,2))),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('Classifier',RandomForestClassifier())  # train on TF-IDF vectors w/ Naive Bayes classifier

     ])

In [ ]:
pipeline.fit(X= msg_train,y=label_train)

In [ ]:
predictions = pipeline.predict(msg_test)

In [ ]:
from sklearn import metrics

In [ ]:
# Print the overall accuracy
print(f"The project accuracy is {round(metrics.accuracy_score(label_test,predictions),3)*100} %")

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(label_test,predictions))